# snippy analysis

In [1]:
import os
import re
import pandas as pd
import seaborn as sns
from Bio import *
import matplotlib.pyplot as plt
import warnings
import numpy as np
import pandas as pd
from Bio import SeqIO
import traceback
from adjustText import adjust_text

In [2]:
def read_vcf_file(filepath): 
    with open(filepath + '/' + 'snps_new.csv', 'w') as f:
        f.write('')
    with open(filepath + '/' + 'snps.vcf', 'r') as f:
        for line in f: 
            if line[0] != '#':
                with open(filepath + '/' + 'snps_new.csv', 'a') as f:
                    f.write(line)  
    columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAl', 'FILTER', 'INFO', 'FORMAT', "filename"]              
    df = pd.read_csv(filepath + '/'+ 'snps_new.csv', sep = '\t', names = columns)
    df = df.drop(['INFO', 'FORMAT', 'filename', 'FILTER'], axis = 1)
    df.to_csv(filepath + '/' + 'snps_new.csv', index = False)
    return df

## read in the snps

In [3]:
ID_df = pd.read_csv(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\anti defence gene bioinformatics\ID' + '/' + 'ID.csv', dtype={'ID':'str'})
ID_df = ID_df.dropna()
ID_df.columns = ['Job', 'Owner', "ID", "phage", "Num contigs", "Size (bp)"]
ID_df = ID_df.reindex(['ID', 'Size (bp)', 'phage'], axis =1)
ID_df.loc[:, 'phage'] = list(ID_df.loc[:, 'phage'].str.extract('(?<=FB)(.*)').loc[:, 0])
ID_df.loc[:, 'ID'] = ID_df.loc[:, 'ID'].astype('str')

In [4]:
#prints all the folders
morphology_df = pd.read_csv("C:/Users/Halewijn/OneDrive/Documenten/third year/BEP/thesis/morphology data" + "/condensed morphology data.csv")
filelist = os.listdir(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\input_snippy_data')
folders_list = []
for filename in filelist:
    if(filename.endswith('inter') or filename.endswith('intra')):
         folders_list = folders_list + [filename]

Here, we extract information out of the foldernames

In [5]:
extr_information = pd.Series(folders_list).str.extract("(?<=myo_)(.*)").replace(np.NaN, '') + pd.Series(folders_list).str.extract("(?<=podo_)(.*)").replace(np.NaN, '')
extr_information = extr_information.rename({0: 'inter or intra + number'}, axis = 1)
extr_information.loc[:, 'family'] = extr_information.loc[:, 'inter or intra + number'].str.extract("(.*?)(?=_i)").replace(np.NaN, 0)
extr_information.loc[:, 'inter or intra'] =extr_information.loc[:, 'inter or intra + number'].str.extract("(?=i)(.*)").replace(np.NaN, 0)
extr_information = extr_information.drop('inter or intra + number', axis = 1)
extr_information.loc[:, 'reference'] = pd.Series(folders_list).str.extract("(Pa.*[0-9].r)")
extr_information.loc[:, 'reference'] = extr_information.loc[:, 'reference'].str[:-1]
indexes = extr_information.loc[:, 'reference'].str[-1] == '_'
extr_information.loc[indexes, 'reference'] = extr_information.loc[:, 'reference'].str[0:-1]
extr_information.loc[:,'reads'] = pd.Series(folders_list).str.extract("(?<=reads)(.*?)(?=m|p|s)")
extr_information.loc[:, 'reads'] =  extr_information.loc[:, 'reads'].str.replace('_', '')
extr_information.loc[:, 'foldername'] = pd.Series(folders_list) 

Here, we extract the snps.raw and paste the information from previously in the dataframe. At the end, the dataframes are concatenated

In [8]:
big_df = []
zero_reads_array = []
zero_reference_array = []
for ii in range(extr_information.shape[0]):
    filepath = r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\input_snippy_data' + '/' + str(extr_information.loc[ii, 'foldername'])
    try: 
        read_vcf_file(filepath)
        df = pd.read_csv(filepath + '/' + 'snps_new.csv')
        df.loc[:, 'family'] = [extr_information.loc[ii, 'family']]*df.shape[0]
        df.loc[:, 'inter or intra'] = [extr_information.loc[ii, 'inter or intra']]*df.shape[0]
        df.loc[:, 'reference'] = [extr_information.loc[ii, 'reference']]*df.shape[0]
        df.loc[:, 'reads'] = [extr_information.loc[ii, 'reads']]*df.shape[0]
        big_df = big_df + [df]
        if df.shape[0] == 0: 
            continue
            # print(extr_information.loc[ii, 'reference'])
            # print(extr_information.loc[ii, 'reads'])

    except: 
        print(str(extr_information.loc[ii, 'foldername']))
big_big_df = pd.concat(big_df)
#because the position is different if we're replacing the bases later
big_big_df.loc[:, 'POS'] = big_big_df.loc[:, 'POS'] -1


In [10]:
num_snps_df = big_big_df[['reads', 'reference']].groupby(['reads', 'reference'], as_index = False).size().pivot_table(index = 'reads', columns = 'reference', values = 'size').fillna(0).stack().reset_index().rename({0: 'count'}, axis = 1)
filt_no_snps_df = num_snps_df[num_snps_df.loc[:, 'count'] < 2]
filt_no_snps_df[filt_no_snps_df.loc[:, 'reads'] != filt_no_snps_df.loc[:, 'reference']]

#phage sizes 
genome_size = pd.read_csv('final_phage_genome_size.csv')

filt_no_snps_df = pd.merge(pd.merge(filt_no_snps_df, genome_size, left_on = 'reference', right_on='Phages').drop('Phages', axis = 1).rename({'Genome size (bp)': 'Size reference'}, axis = 1), genome_size, left_on = 'reads', right_on= 'Phages').drop('Phages', axis = 1).rename({'Genome size (bp)': 'Size reads'}, axis = 1)
#pd.merge(filt_no_snps_df, ID_df[['phage', 'Size (bp)']], left_on='reference', right_on = 'phage', copy= False)
filt_filt_no_snps_df =  filt_no_snps_df[filt_no_snps_df.loc[:, 'reads'] != filt_no_snps_df.loc[:, 'reference']]
filt_filt_no_snps_df.loc[:, 'size difference'] = filt_filt_no_snps_df.loc[:, 'Size reference'] - filt_filt_no_snps_df.loc[:, 'Size reads']
filt_filt_no_snps_df.to_csv('checkout.csv')

In [12]:
#sns.clustermap(big_big_df.groupby(['reference', 'reads'], as_index = False).size().pivot_table(index = 'reads', columns='reference', values='size').fillna(0))

In [14]:
amount_of_snps = big_big_df.groupby(['reference', 'reads'], as_index = False).size().sort_values('size', ascending = True)
amount_of_snps = pd.merge(amount_of_snps, morphology_df, left_on = 'reference', right_on='phage').rename({'morphology': 'morphology reference'}, axis = 1)
amount_of_snps = pd.merge(amount_of_snps, morphology_df, left_on = 'reads', right_on = 'phage').rename({'morphology': 'morphology reads'}, axis = 1)
#amount_of_snps[(amount_of_snps.loc[:, 'morphology reads'] == 'Myoviridae') & (amount_of_snps.loc[:, 'morphology reference'] == 'Myovirdae')]
a = amount_of_snps[(amount_of_snps.loc[:, 'morphology reads'] == 'Myoviridae') & (amount_of_snps.loc[:, 'morphology reference'] == 'Myoviridae')].sort_values('size').head(60)

#amount_of_snps.sort_values('size', ascending = True).head(50)
# amount_of_snps[amount_of_snps.loc[:, 'reads'] == 'Pa29']
# amount_of_snps.loc[:, 'reference'].unique()
# amount_of_snps[()]
#amount_of_snps[(amount_of_snps.loc[:, 'morphology reads'] == 'Myoviridae') & (amount_of_snps.loc[:, 'morphology reference'] == 'Myoviridae')].sort_values('size').head(60)
#a.loc[:, 'reads'].unique()

In [15]:
size_intra_inter = big_big_df.groupby('inter or intra', as_index = False).count().loc[:, 'reads']
percentage_intra = (size_intra_inter[1]/size_intra_inter[0])*100
percentage_intra

4.0274618357223115

## Add the morphology

add the morphology to the snp dataframe. From the reference, but also from the reads. The dataframe is saved as morph_big_big_df

In [17]:
morph_big_big_df = pd.merge(big_big_df, morphology_df,left_on='reference', right_on = 'phage')
morph_big_big_df = morph_big_big_df[['POS', 'REF', 'ALT', 'reference', 'reads', 'morphology']].rename({'morphology': 'morphology reference'}, axis = 1)
morph_big_big_df = pd.merge(morph_big_big_df, morphology_df,left_on='reads', right_on = 'phage').rename({'morphology': 'morphology reads'}, axis = 1)

In [18]:
morph_big_big_df.to_csv('morph_big_big_df.csv')

## Add the ID

Here, we get the positions of each SNP, and also the ID of the gbk file. The dataframe is saves as ID_morph_big_big_df

In [22]:
#secondly, import the ID df, because the all_snippy_dataframe does not have the phage ID with it. make sure to merge at the end with the latest ID
ID_df = pd.read_csv(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\anti defence gene bioinformatics\ID' + '/' + 'ID.csv', dtype={'ID':'str'})
ID_df = ID_df.dropna()
ID_df.columns = ['Job', 'Owner', "ID", "phage", "Num contigs", "Size (bp)"]
ID_df = ID_df.reindex(['ID', 'Size (bp)', 'phage'], axis =1)
ID_df.loc[:, 'phage'] = list(ID_df.loc[:, 'phage'].str.extract('(?<=FB)(.*)').loc[:, 0])
ID_df.loc[:, 'ID'] = ID_df.loc[:, 'ID'].astype('str')

#merge them
ID_morph_big_big_df = pd.merge(morph_big_big_df, ID_df[['ID', 'phage']], left_on='reference', right_on = 'phage').rename({'ID':'ID of reference'}, axis = 1).drop(['phage_x', 'phage_y'], axis = 1)

In [23]:
ID_morph_big_big_df.to_csv('ID_morph_big_big_df.csv')

## import the genic regions

All the gene regions, with their respective annotations are make here and saved as big_intergenic_df.csv

In [25]:
#first, make the genic dataframe
big_intergenic_df = []
phage_gbks_filepath = r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\intergenic regions\phage gbk'
phage_gbks_filenames = os.listdir(phage_gbks_filepath)
for phage_gbk_filename in phage_gbks_filenames: 
    dictionary = {'gene start': [], 'gene stop': [], 'complement': [], 'protein translation': [], 'protein annotation': [], 'product': []}
    for seq_record in SeqIO.parse(phage_gbks_filepath + '/' + phage_gbk_filename, 'genbank'): 
        for seq_feature in seq_record.features:
            if seq_feature.type == 'CDS': 
                start = seq_feature.location.start
                stop = seq_feature.location.end
                complement = seq_feature.location.strand
                translation = seq_feature.qualifiers['translation']
                annotation = seq_feature.qualifiers['db_xref']
                product = seq_feature.qualifiers['product']
                dictionary['gene start'].append(int(start))
                dictionary['gene stop'].append(int(stop))
                dictionary['complement'].append(complement)
                dictionary['protein translation'].append(translation)
                dictionary['protein annotation'].append(annotation)
                dictionary['product'].append(product)
    intergenic_df = pd.DataFrame(dictionary)
    intergenic_df.loc[:, 'ID'] = [phage_gbk_filename[:-4]]*intergenic_df.shape[0]
    big_intergenic_df = big_intergenic_df + [intergenic_df]  
    
big_intergenic_df = pd.concat(big_intergenic_df, ignore_index=True)

#also merge it with the morphology df
big_intergenic_df = pd.merge(big_intergenic_df, ID_df[['ID', 'phage']])
big_intergenic_df.to_csv('big_intergenic_df.csv', index = False)

## Find where each snp belongs to

This code takes a while to run. Here, we go over each snp position for a certain reference. If its find a hit, it gets saved, together with the annotation etc. The dataframes is later concatenated and saved as snp_in_gene_df 

In [27]:
references = ID_morph_big_big_df.loc[:, 'reference'].unique()
all_hits = []
for i in range(len(references)): 
    reference = references[i]
    print(reference)
    ref_snp_df = ID_morph_big_big_df[ID_morph_big_big_df.loc[:, 'reference'] == reference][['POS', 'reads']].reset_index(drop = True)
    ref_genic_df = big_intergenic_df[big_intergenic_df.loc[:, 'phage'] == reference][['gene start', 'gene stop' , 'protein annotation', 'product', 'complement', 'protein translation']].reset_index(drop = True)
    snp_length = ref_snp_df.shape[0]
    genic_length = ref_genic_df.shape[0]
    for ii in range(snp_length): 
        position = ref_snp_df.loc[ii, 'POS']
        reads = ref_snp_df.loc[ii, 'reads']
        hit = ref_genic_df[((ref_genic_df.loc[:, 'gene start'] <= position) & (ref_genic_df.loc[:, 'gene stop'] >= position))]
        hit.loc[:, 'position'] = [position]*hit.shape[0]
        hit.loc[:, 'reads'] = [reads]*hit.shape[0]
        #hit.loc[:, 'complement'] = [complement]*hit.shape[0]
        
        hit.loc[:, 'reference'] = [reference]*hit.shape[0]
        all_hits = all_hits + [pd.DataFrame(hit)]
    # big_hit = pd.concat(big_hit, ignore_index=True)
    # big_hit.loc[:, 'phage'] = [reference]*big_hit.shape[0]
    # big_big_hit = big_big_hit + [big_hit]

Pa10


C:\Users\Halewijn\anaconda3\lib\site-packages\pandas\core\indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


Pa14
Pa21
Pa24


KeyboardInterrupt: 

In [ ]:
pos_gene_translation = pd.concat(all_hits)
snp_in_gene_df = pd.merge(ID_morph_big_big_df, pos_gene_translation, left_on = ['POS', 'reference', 'reads'], right_on=['position', 'reference', 'reads']).drop(['position'], axis = 1)
snp_in_gene_df.to_csv('snp_in_gene_df.csv', index = False)

In [ ]:
# pos_gene_translation = pd.concat(big_big_hit, ignore_index=True)
# pos_gene_translation = pd.merge(ID_morph_big_big_df, pos_gene_translation[['protein translation', 'position', 'phage', 'reads', 'gene start', 'gene stop', 'complement']], left_on = ['POS', 'reference', 'reads'], right_on=['position', 'phage', 'reads'])
# pos_gene_translation =  pos_gene_translation[['POS', 'REF', 'gene start', 'gene stop','ALT', 'reference', 'reads', 'morphology reference', 'morphology reads', 'ID of reference', 'protein translation', 'complement']]

## filter step

filter on morphology, the fact that we can only have 1 snp, and that the protein has to start with M. The dataframe is saved as filtered_snp_df.csv

In [28]:
snp_in_gene_df = pd.read_csv('snp_in_gene_df.csv')
filtered_snp_df = snp_in_gene_df[(snp_in_gene_df.loc[:, 'REF'].str.len() == 1) & (snp_in_gene_df.loc[:, 'morphology reference'] == 'Myoviridae')
                                & (snp_in_gene_df.loc[:, 'morphology reads'] == 'Myoviridae')].reset_index(drop = True)
filtered_snp_df = filtered_snp_df[snp_in_gene_df.loc[:,'protein translation'].astype('str').str[2] == 'M']
filtered_snp_df =  pd.merge(filtered_snp_df.drop('ID of reference', axis =1 ), ID_df, left_on='reference', right_on='phage')
filtered_snp_df.to_csv('filtered_snp_df.csv')

C:\Users\Halewijn\AppData\Local\Temp/ipykernel_664/4060901585.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_snp_df = filtered_snp_df[snp_in_gene_df.loc[:,'protein translation'].astype('str').str[2] == 'M']


## get the DNA sequence, next to each row, and make the mutations

Here, we go to one of the last steps, in which the adaptation is made to the protein. Be carefull here: the adaption is made on the read sequence, not on the reference sequence. This code takes a while to run. It is eventually saved in two arrays: the error array and the alt_protein array. The result is a dataframe with all mutations, their effect on the protein and the exceptions if there are. The dataframe is saved as: final_df.csv

In [ ]:
filtered_snp_df = pd.read_csv('filtered_snp_df.csv', dtype={'ID':'str'})
filtered_snp_df.loc[:, 'ID'].unique()

In [ ]:
synonymous = []
synonymous_array = []
error_array = []
alt_protein_array = []
alt_sequence_array = []
error_dictionary = {}
#for ii in range(filtered_snp_df.shape[0]): 
i = 0
for ii in range(filtered_snp_df.shape[0]):
    i = i + 1 
    if i == 10000: 
        print(str(i))
        i = 0
    row = ii
    phage_genbank_filepath = r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\intergenic regions\phage gbk'
    gene_start = filtered_snp_df.loc[row, 'gene start']
    gene_stop = filtered_snp_df.loc[row, 'gene stop']
    ID = filtered_snp_df.loc[row, 'ID']
    REF = filtered_snp_df.loc[row, 'REF']
    POS = filtered_snp_df.loc[row, 'POS']
    ALT = filtered_snp_df.loc[row, 'ALT']
    complement = filtered_snp_df.loc[row, 'complement']
    original_protein = filtered_snp_df.loc[row, 'protein translation'][0]
    for seq_record in SeqIO.parse(phage_genbank_filepath + '/' + ID + '.gbk', 'genbank'): 
        sequence = seq_record.seq
        gene = sequence[gene_start:gene_stop]
        pos_relative_to_gene = POS - gene_start
        alt_gene = gene[:pos_relative_to_gene] + ALT + gene[pos_relative_to_gene+1:]
        if complement == -1: 
            alt_gene = alt_gene.reverse_complement()
        error = False
        try: 
            alt_protein = alt_gene.translate(cds = True)
        except Exception as x: 
            error = x
            alt_protein = alt_gene.translate(cds = False)
        error_array = error_array + [error]
        alt_sequence_array = alt_sequence_array + [alt_gene]
        alt_protein_array = alt_protein_array + [alt_protein]
        #alt_protein_array = alt_protein_array + [alt_protein]

In [ ]:
filtered_snp_df.loc[:,'altered sequence'] = alt_sequence_array
filtered_snp_df.loc[:, 'error'] = error_array
filtered_snp_df.loc[:, 'altered protein'] = alt_protein_array
filtered_snp_df.to_csv('final_df.csv',index = False)

## This time, include the fact that multiple mutations occur

Is there the need to combine all the mutations? 

In [ ]:
filtered_df_snp = pd.read_csv('filtered_snp_df.csv', dtype={'ID':'str'}).drop('Unnamed: 0', axis = 1)
filtered_df_snp.loc[:, 'size protein'] = filtered_df_snp.loc[:, 'protein translation'].str[2:-2].str.len()
#sns.histplot(data = filtered_df_snp.groupby(['protein annotation'],as_index = False).size(), x = 'size')
#filtered_df_snp.groupby(['protein annotation'],as_index = False)[]

In [ ]:
#filtered_df_snp.groupby(['protein annotation'], as_index = False)
num_mutations_vs_size_df = filtered_df_snp.groupby(['protein annotation'], as_index = False)['size protein'].mean()
num_mutations_vs_size_df.loc[:, 'num mutations'] =  filtered_df_snp.groupby(['protein annotation'], as_index = False).size().loc[:, 'size']
num_mutations_vs_size_df.loc[:, 'ratio'] = num_mutations_vs_size_df.loc[:, 'size protein']/num_mutations_vs_size_df.loc[:, 'num mutations']
num_mutations_vs_size_df.loc[:, 'product'] = filtered_df_snp.groupby('protein annotation', as_index = False)['product'].agg(pd.Series.mode).loc[:, 'product']

#find out the relation between mutations in polymerase and mutations in the rest of the genome
DNA_polymerase_df = filtered_df_snp[(filtered_df_snp.loc[:, 'product'] == "['DNA polymerase III alpha subunit (EC 2.7.7.7)']") 
                                   ]
gr_DNA_polymerase_df = DNA_polymerase_df.groupby('protein annotation', as_index = False).size().rename({"size": 'mutations in polymerase'}, axis = 1)
gr_DNA_polymerase_df.loc[:, 'ID'] = DNA_polymerase_df.groupby('protein annotation', as_index = False)['ID'].agg(pd.Series.mode).loc[:, 'ID']
gr_DNA_polymerase_df.loc[:, 'tot mutations'] = filtered_df_snp.groupby('ID', as_index = False)['ID'].size().loc[:, 'size']
gr_DNA_polymerase_df.loc[:, 'mutations except DNA pol'] = gr_DNA_polymerase_df.loc[:, 'tot mutations'] - gr_DNA_polymerase_df.loc[:, 'mutations in polymerase']
gr_DNA_polymerase_df = pd.merge(gr_DNA_polymerase_df, ID_df[['phage', 'ID']])
#scatterplot
x = gr_DNA_polymerase_df.loc[:, 'mutations in polymerase']
y = gr_DNA_polymerase_df.loc[:, 'mutations except DNA pol']
labels = gr_DNA_polymerase_df.loc[:, 'phage']

fig, axes = plt.subplots(2,3, figsize = (10,7))
sns.histplot(data = num_mutations_vs_size_df, x ='num mutations', ax = axes[0][0])
sns.histplot(data = num_mutations_vs_size_df, x='num mutations', y = 'size protein', ax = axes[0][1])
sns.regplot(data = num_mutations_vs_size_df, x = 'num mutations', y='size protein', ax = axes[0][2], scatter_kws={'s':0.5}, ci = False)
sns.histplot(data = num_mutations_vs_size_df[num_mutations_vs_size_df.loc[:, 'ratio'] < 1.5], x = 'ratio', bins = 50, ax = axes[1][0])
sns.regplot(data = gr_DNA_polymerase_df, x = 'mutations in polymerase', y = 'mutations except DNA pol', ax = axes[1][1], ci = False, scatter_kws={'s':5})
sns.scatterplot(x=x, y=y, ax = axes[1][2])
texts = [plt.text(x_pos, y_pos, f'{l}') for (x_pos, y_pos, l) in zip(x, y, labels)]
adjust_text(texts)

plt.tight_layout()
plt.show()

So yes, there is a need for this. What we can also see is that the amount of mutations does not perfectly scale linearly with the size of the protein. Which I think is funny. What is the regression coefficient? The slope is almost 1. In the beginning however, the slope is steeper. Indicating that for small proteins, you actually have a lot of mutations. The last plot is maybe not so right after all. The sum of mutations namely really depends on what phages you compare to each other, so maybe I need to filter on that. So to just take a phage as reference, assuming this one has no mutations

In [ ]:
DNA_polymerase_df = filtered_df_snp[(filtered_df_snp.loc[:, 'product'] == "['DNA polymerase III alpha subunit (EC 2.7.7.7)']")]
#DNA_polymerase_df.groupby('reads', as_index = False).size().sort_values('size')
# gr_DNA_polymerase_df = DNA_polymerase_df.groupby('protein annotation', as_index = False).size().rename({"size": 'mutations in polymerase'}, axis = 1)
# gr_DNA_polymerase_df.loc[:, 'ID'] = DNA_polymerase_df.groupby('protein annotation', as_index = False)['ID'].agg(pd.Series.mode).loc[:, 'ID']
# gr_DNA_polymerase_df.loc[:, 'tot mutations'] = filtered_df_snp.groupby('ID', as_index = False)['ID'].size().loc[:, 'size']
# gr_DNA_polymerase_df.loc[:, 'mutations except DNA pol'] = gr_DNA_polymerase_df.loc[:, 'tot mutations'] - gr_DNA_polymerase_df.loc[:, 'mutations in polymerase']
# gr_DNA_polymerase_df = pd.merge(gr_DNA_polymerase_df, ID_df[['phage', 'ID']])

In [ ]:
reference_phages= DNA_polymerase_df.loc[:, 'reference'].unique()
df_array = []
for ii in range(len(reference_phages)): 
    reference = reference_phages[ii]
    ref_DNA_polymerase_df = DNA_polymerase_df[DNA_polymerase_df.loc[:, 'reference'] == reference]
    gr_ref_DNA_polymerase_df = ref_DNA_polymerase_df.groupby('reads').size().reset_index()
    gr_ref_DNA_polymerase_df.loc[:, 'reference'] = [reference]*gr_ref_DNA_polymerase_df.shape[0]
    df_array = df_array + [gr_ref_DNA_polymerase_df]
heatmap_df = pd.concat(df_array)
sns.heatmap(heatmap_df.pivot_table(index = 'reads', columns='reference', values=0).fillna(0))

make the algorithm to induce the multiple mutations

## this is for another section

In [ ]:
#in this kernel, I want to find out where exactly the stop codon is. Is it in the middle, or more at the end?
wrong_start_stop_df = [filtered_snp_df.loc[:, 'error'] !='False']
wrong_start_df = wrong_start_stop_df[wrong_start_stop_df.loc[:, 'error'].str.contains('is not a start codon')].reset_index(drop = True)
# wrong_start_df
#wrong_start_df.groupby('error', as_index = False).count().sort_values('POS', ascending = False)

alt_protein_array = []
#for ii in range(wrong_start_df.shape[0]): 
i = 0
for ii in range(wrong_start_df.shape[0]):
    i = i + 1 
    if i == 10000: 
        print(str(i))
        i = 0
    row = ii
    phage_genbank_filepath = r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\intergenic regions\phage gbk'
    gene_start = wrong_start_df.loc[row, 'gene start']
    gene_stop = wrong_start_df.loc[row, 'gene stop']
    ID = wrong_start_df.loc[row, 'ID']
    REF = wrong_start_df.loc[row, 'REF']
    POS = wrong_start_df.loc[row, 'POS']
    ALT = wrong_start_df.loc[row, 'ALT']
    complement = wrong_start_df.loc[row, 'complement']
    original_protein = wrong_start_df.loc[row, 'protein translation'][0]
    for seq_record in SeqIO.parse(phage_genbank_filepath + '/' + ID + '.gbk', 'genbank'): 
        a = 1
        
        sequence = seq_record.seq
        gene = sequence[gene_start:gene_stop]
        pos_relative_to_gene = POS - gene_start
        alt_gene = gene[:pos_relative_to_gene] + ALT + gene[pos_relative_to_gene+1:]
        if complement == -1: 
            alt_gene = alt_gene.reverse_complement()
        alt_protein = alt_gene.translate()
        alt_protein_array = alt_protein_array + [alt_protein]
        # error = False
        # try: 
        #     alt_protein = alt_gene.translate(cds = True)
        # except Exception as x: 
        #     error = x
        # error_array = error_array + [error]
        # synonymous_array = synonymous_array + [alt_protein]

In [ ]:
# wrong_start_df.loc[:, 'altered protein'] = alt_protein_array
# wrong_start_df.to_csv('test.csv')

In [ ]:
wrong_start_df.loc[:, 'length of original gene'] = wrong_start_df.loc[:, 'altered protein'].str.len()
wrong_start_df.loc[:, 'new start pos'] = wrong_start_df.loc[:, 'altered protein'].astype('str').str.find('M')
wrong_start_df.loc[:, 'percentage of gene cut off'] = wrong_start_df.loc[:, 'new start pos']/wrong_start_df.loc[:, 'length of original gene']*100
wrong_start_df = wrong_start_df.drop(['length of original gene', 'new start pos'], axis = 1)
wrong_start_df.to_csv('wrong_start_df.csv', index = False)

In [ ]:
a = pd.DataFrame(synonymous)
a.loc[:, 'equals false'] = (a.loc[:, 0] == False)
a[a.loc[:, 'equals false'] == True]

In [ ]:
gene == alt_gene

In [ ]:
for seq_record in SeqIO.parse(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\intergenic regions\phage gbk'
                             + '/' + '287.19287.gbk', 'genbank'): 
    a = seq_record.seq
    print(a[286
           ])

In [ ]:
#now make an array for every startstop combination
array_dict = {}
for ii in range(big_intergenic_df.shape[0]): 
    start = big_intergenic_df.loc[ii, 'gene start']
    stop = big_intergenic_df.loc[ii, 'gene stop']
    array_dict[ii] = np.arange(start, stop, 1)
#array_dict.keys()

big_intergenic_df.loc[:, 'start stop array'] = array_dict

In [ ]:
dictionary = {'gene start': [], 'gene stop': []}
for seq_record in SeqIO.parse(phage_gbks_filepath + '/' + '287.19293.gbk', 'genbank'): 
    for seq_feature in seq_record.features:
        if seq_feature.type == 'CDS': 
            start = seq_feature.location.start
            stop = seq_feature.location.end
            dictionary['gene start'].append(int(start))
            dictionary['gene stop'].append(int(stop))
intergenic_df = pd.DataFrame(dictionary)
intergenic_df.loc[:, 'ID'] = [phage_gbk_filename[:-4]]*intergenic_df.shape[0]

In [ ]:
csv_files = os.listdir(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\intergenic regions\saved_intergenic_df')
intergenic_regions_csvs = []
for csv_file in csv_files: 
    if csv_file.endswith('_region.csv'): 
        intergenic_regions_csvs = intergenic_regions_csvs + [csv_file]
        
#get the ID name out
csv_file_df = pd.DataFrame(intergenic_regions_csvs).rename({0: 'filename'}, axis = 1)
ID_array = list(csv_file_df.loc[:, 'filename'].str.extract('(.*?)(?<= )').replace(np.NaN, ' ').loc[:, 0])
ID_array = [ x[:-1] for x in ID_array ]
csv_file_df.loc[:, 'ID'] = ID_array
indexes_remaning = csv_file_df[csv_file_df.loc[:, 'ID'] == ''].index
ID_array_remaining = list(csv_file_df.iloc[indexes_remaning].loc[:, 'filename'].str.extract('(.*?)(?=intergenic)').loc[:,0])
csv_file_df.loc[indexes_remaning, 'ID'] = ID_array_remaining

# #also do the import the ID dataset, so that we can merge on phage name
# ID_df = pd.read_csv(r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\anti defence gene bioinformatics\ID' + '/' + 'ID.csv')
# ID_df = ID_df.dropna()
# ID_df.columns = ['Job', 'Owner', "ID", "phage", "Num contigs", "Size (bp)"]
# ID_df = ID_df.reindex(['ID', 'Size (bp)', 'phage'], axis =1)
# ID_df.loc[:, 'phage'] = list(ID_df.loc[:, 'phage'].str.extract('(?<=FB)(.*)').loc[:, 0])
# ID_df.loc[:, 'ID'] = ID_df.loc[:, 'ID'].astype('str')

# #merge the ID and the csv dataset
# csv_file_df = pd.merge(csv_file_df, ID_df).reset_index(drop = True)


# #now that everything is ready, we can start importing the intergenic dataset, we need to loop over all the filenames in our csv_file_df
# input_filepath = r'C:\Users\Halewijn\OneDrive\Documenten\third year\BEP\thesis\mutational analysis\intergenic regions\saved_intergenic_df'
# big_intergenic_df = []
# for ii in range(csv_file_df.shape[0]):
#     input_filename = csv_file_df.loc[ii, 'filename']
#     intergenic_df = pd.read_csv(input_filepath + '/' + input_filename).rename({'unnamed: 0': 'POS'}, axis = 1)
#     #phage_name = csv_file_df.loc[ii, 'phage']
#     #print(phage_name)
#     #intergenic_df.loc[:, 'phage'] = [phage_name]*intergenic_df.shape[0]
#     big_intergenic_df = big_intergenic_df + [intergenic_df]
    
# big_intergenic_df = pd.concat(big_intergenic_df).rename({'Unnamed: 0': 'POS'}, axis = 1)
# #big_intergenic_df.loc[:, 'ALT evidence'] = big_intergenic_df.loc[:, 'EVIDENCE'].str.extract('(?<=:)(.*)(?= )').loc[0]

# #now merge 
# #intergenic_big_big_df = pd.merge(big_big_df, big_intergenic_df, left_on = ['POS', 'reference'], right_on = ['POS', 'reference'])
# ref_big_big_df = pd.merge(big_big_df, morphology_df, left_on = 'reference', right_on='phage').rename({'morphology': 'morphology reference'}, axis = 1)
# read_ref_big_big_df = pd.merge(ref_big_big_df, morphology_df, left_on = 'reads', right_on='phage').rename({'morphology': 'morphology read'}, axis = 1)

# #big
# #intergenic_big_big_df.to_csv('big_snp_df.csv', index = False)
# #intergenic_big_big_df.to_csv('snippy_df.csv', index = False)
# read_ref_big_big_df.to_csv('snippy_df.csv', index = False)

In [ ]:
intergenic_regions_csvs

In [ ]:
big_intergenic_df

In [ ]:
intergenic_big_big_df

In [ ]:
phage = 'Pa12'
type_ = 'snp'
inter_or_intra = 'intra'
morphology = 'Myoviridae'
intra_df_snp = intergenic_big_big_df[(intergenic_big_big_df.loc[:, 'reference'] == phage) & (intergenic_big_big_df.loc[:, 'TYPE'] == type_)
                     & (intergenic_big_big_df.loc[:, 'inter or intra'] == inter_or_intra)]

In [ ]:
evidence_data = intra_df_snp.loc[:, 'ALT evidence']
log_evidence_data= np.log10(evidence_data)
minimum = log_evidence_data.min()
maximum = log_evidence_data.max()
print(minimum)
print(maximum)
#x_tick_array = np.arange(minimum, maximum, 100)
sns.displot(log_evidence_data)
#plt.xticks([])
plt.show()

In [ ]:
phage = 'Pa10'
type_ = 'snp'
inter_or_intra = 'intra'
morphology = 'Myoviridae'
intra_df_snp = intergenic_big_big_df[(intergenic_big_big_df.loc[:, 'morphology'] == morphology) & (intergenic_big_big_df.loc[:, 'TYPE'] == type_)
                     & (intergenic_big_big_df.loc[:, 'inter or intra'] == inter_or_intra)]

In [ ]:
evidence_data = intra_df_snp.loc[:, 'ALT evidence']
log_evidence_data= np.log10(evidence_data)
minimum = log_evidence_data.min()
maximum = log_evidence_data.max()
print(minimum)
print(maximum)
#x_tick_array = np.arange(minimum, maximum, 100)
sns.displot(log_evidence_data)
#plt.xticks([])
plt.show()

In [ ]:
filled_intra_df_snp = intra_df_snp
snp_positions = list(filled_intra_df_snp.loc[:, 'POS'])
unfilled_positions = list(set(np.arange(0,max(snp_positions),1)).symmetric_difference(set(snp_positions)))

dictionary = {}
# for ii in range(len(unfilled_positions)): 

number_of_unfilled_positions = len(unfilled_positions)

#CHROM_array
CHROM_array = [filled_intra_df_snp.iloc[0, 0]]*number_of_unfilled_positions
#POS_array
POS_array = []
for ii in range(len(unfilled_positions)): 
    POS_array = POS_array + [unfilled_positions[ii]]
#TYPE
TYPE_array = ['no snp']*number_of_unfilled_positions
#REF 
REF_array = [np.NaN]*number_of_unfilled_positions
#ALS 
ALT_array = [np.NaN]*number_of_unfilled_positions
#EVIDENCE
EVIDENCE_array = [np.NaN]*number_of_unfilled_positions
#family
family_array = [np.NaN]*number_of_unfilled_positions
#inter or intra
inter_or_intra_array = [np.NaN]*number_of_unfilled_positions
#reference
reference_array = [np.NaN]*number_of_unfilled_positions
#reads
reads_array = [np.NaN]*number_of_unfilled_positions




dictionary['CHROM'] = CHROM_array
dictionary["POS"] = POS_array
dictionary['TYPE'] = TYPE_array
dictionary['REF'] = REF_array
dictionary['ALT'] = ALT_array
dictionary['EVIDENCE'] = EVIDENCE_array
dictionary['inter or intra'] = inter_or_intra_array
dictionary['reference'] = reference_array
dictionary['reads'] = reads_array

In [ ]:
frame = 50
max_frame = 5
filled_intra_df_snp = filled_intra_df_snp.append(pd.DataFrame(dictionary)).sort_values('POS')
filled_intra_df_snp.loc[:, 'snp'] = list(filled_intra_df_snp.loc[:, 'TYPE'] == 'snp')
filled_intra_df_snp.loc[:, 'moving sum of snps'] =list(filled_intra_df_snp.loc[:, 'snp'].rolling(frame).sum())
filled_intra_df_snp.loc[:, 'moving max of moving sum of snps'] =list(filled_intra_df_snp.loc[:, 'moving sum of snps'].rolling(max_frame).max())


In [ ]:
filled_intra_df_snp

In [ ]:
sns.scatterplot(data = filled_intra_df_snp, x = 'POS', y = 'inter or intra')
plt.xlim(0,10000)

In [ ]:
sns.scatterplot(data = filled_intra_df_snp, x = 'POS', y = 'moving max of moving sum of snps', hue = 'intergenic')
plt.show()

In [ ]:
hist_df = filled_intra_df_snp[filled_intra_df_snp.loc[:, 'moving sum of snps'] > 12].reset_index()
sns.histplot(data = hist_df, x = 'POS', bins=200, kde=True)
plt.show()

In [ ]:
big_big_df[(big_big_df.loc[:, 'inter or intra'] == 'intra') & (big_big_df.loc[:, 'family'] == 2)].shape[0]

In [ ]:
big_big_df.loc[:, 'family'].unique()